## Preprocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
# read dataset
data = pd.read_csv("data/ds_crmtt.csv")
data.head()

,tanggal,jenis,qty,harga,jumlah
0,5/8/2023,CRMTT,"1,000.00","23,000.00","23,000,000.00"
1,5/10/2023,CRMTT,958.00,"23,000.00","22,034,000.00"
2,5/11/2023,CRMTT,"1,042.00","23,000.00","23,966,000.00"
3,5/11/2023,CRMTT,508.00,"30,000.00","15,240,000.00"
4,5/13/2023,CRMTT,"1,291.00","30,000.00","38,730,000.00"


In [33]:
# membaca informasi data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   tanggal  210 non-null    object
 1   jenis    210 non-null    object
 2   qty      210 non-null    object
 3   harga    210 non-null    object
 4   jumlah   210 non-null    object
dtypes: object(5)
memory usage: 8.3+ KB


In [44]:
# copy data
df = data.copy()

In [45]:
# mengubah df type
df["tanggal"] = pd.to_datetime(df["tanggal"], format='%m/%d/%Y')

df["harga"] = df["harga"].str.replace(",", "")
df["harga"] = df["harga"].str.replace(" ", "")
df["harga"] = df["harga"].str.replace(".", "")
df["harga"] = df["harga"].astype(int)

df["qty"] = df["qty"].str.replace(",", "")
df["qty"] = df["qty"].str.replace(".", "")
df["qty"] = df["qty"].str.replace(" ", "")
df["qty"] = df["qty"].astype(int)

df["jumlah"] = df["jumlah"].str.replace(",", "")
df["jumlah"] = df["jumlah"].str.replace(" ", "")
df["jumlah"] = df["jumlah"].str.replace(".", "")
df["jumlah"] = df["jumlah"].astype(float)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   tanggal  210 non-null    datetime64[ns]
 1   jenis    210 non-null    object        
 2   qty      210 non-null    int32         
 3   harga    210 non-null    int32         
 4   jumlah   210 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int32(2), object(1)
memory usage: 6.7+ KB


In [47]:
# mengecek missing value
df.isnull().sum()

tanggal    0
jenis      0
qty        0
harga      0
jumlah     0
dtype: int64

In [48]:
# mengecek duplicate value
df.duplicated().sum()

0

In [49]:
# group by tanggal
df_group = df.groupby(["tanggal", "jenis"]).agg({"qty": "sum","harga": "sum","jumlah": "sum"}).reset_index()
df_group.head()

,tanggal,jenis,qty,harga,jumlah
0,2023-05-08,CRMTT,100000,2300000,2.300000e+09
1,2023-05-10,CRMTT,95800,2300000,2.203400e+09
2,2023-05-11,CRMTT,155000,5300000,3.920600e+09
3,2023-05-13,CRMTT,129100,3000000,3.873000e+09
4,2023-05-18,CRMTT,5750,3000000,1.725000e+08


In [55]:
# create data range 
min_date = df_group['tanggal'].min()
max_date = df_group['tanggal'].max()

# buat range data tanggal
date_range = pd.date_range(start=min_date, end=max_date)
df_tanggal= pd.DataFrame(date_range, columns=['tanggal'])

In [57]:
df_tanggal.head()
df.shape

(210, 5)

In [75]:
# menggabungkan dataset
merged_df = pd.merge(df_tanggal, df_group, on='tanggal', how='left')
merged_df.head()

,tanggal,jenis,qty,harga,jumlah
0,2023-05-08,CRMTT,100000.0,2300000.0,2.300000e+09
1,2023-05-09,NaN,NaN,NaN,NaN
2,2023-05-10,CRMTT,95800.0,2300000.0,2.203400e+09
3,2023-05-11,CRMTT,155000.0,5300000.0,3.920600e+09
4,2023-05-12,NaN,NaN,NaN,NaN


In [76]:
# mengisi nilai kosong
merged_df["jenis"] = merged_df["jenis"].fillna("CRMTT")
merged_df["qty"].interpolate(inplace=True)
merged_df["jumlah"].interpolate(inplace=True)
merged_df["harga"].interpolate(inplace=True)
merged_df.head()

C:\Users\Aditya\AppData\Local\Temp\ipykernel_11036\1838387698.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df["qty"].interpolate(inplace=True)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_11036\1838387698.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

,tanggal,jenis,qty,harga,jumlah
0,2023-05-08,CRMTT,100000.0,2300000.0,2.300000e+09
1,2023-05-09,CRMTT,97900.0,2300000.0,2.251700e+09
2,2023-05-10,CRMTT,95800.0,2300000.0,2.203400e+09
3,2023-05-11,CRMTT,155000.0,5300000.0,3.920600e+09
4,2023-05-12,CRMTT,142050.0,4150000.0,3.896800e+09


In [74]:
print(merged_df.isnull().sum())
print(merged_df.duplicated().sum())

tanggal    0
jenis      0
qty        0
harga      0
jumlah     0
dtype: int64
0


In [77]:
# save dataset
merged_df.to_csv("data/ds_crmtt_clean.csv", index=False)